In [1]:
import requests
from urllib.error import URLError, HTTPError
from http.client import IncompleteRead
from time import sleep

In [2]:
# kakao api
APP_KEY = ""
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

In [3]:
def kakao_req(address):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)
    kakao_url_string = str(kakao_url)+ '?query='+ str(address)
    try:
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except HTTPError as e:
        print("502 error caused reply" + e)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except IncompleteRead:
        # Oh well, reconnect and keep trucking
        print("_chunk_size Error")
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    except URLError as e:
        print(e.reason)
        resultXML = requests.get(kakao_url_string, headers=kakao_headers)
    else:
        data= resultXML.json()
        if len(data['documents']) ==0:
            data= '403'
            return data
        else:
            return resultXML

In [4]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = kakao_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [5]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np
from pyspark.sql import SQLContext

In [6]:
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [7]:
basic_folder = './apartment_trade_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)

apartment_trade_202001.csv


In [8]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [9]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- trade_price: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [10]:
df.show()
df.count()

+---+----------+-----------+----+-----+---+--------+-------------+-------+-----+---------------------+-----+----------------------+------------------------------+----------------+
|_c0|build_year|trade_price|year|month|day|    city|     sub_city|   dong| code|                 name|floor|exclusive_private_area|                       address|detailed_address|
+---+----------+-----------+----+-----+---+--------+-------------+-------+-----+---------------------+-----+----------------------+------------------------------+----------------+
|  0|      2019|     58,900|2020|    1|  6|경상남도|창원시 의창구|   중동|48121|창원중동유니시티1단지|   40|               99.1742|  경상남도 창원시 의창구  중동|             790|
|  1|      2019|     52,773|2020|    1| 22|경상남도|창원시 의창구|   중동|48121|창원중동유니시티3단지|   16|               84.7171|  경상남도 창원시 의창구  중동|             786|
|  2|      2019|     54,000|2020|    1| 22|경상남도|창원시 의창구|   중동|48121|창원중동유니시티1단지|   10|               84.1031|  경상남도 창원시 의창구  중동|             790|
|  3|      2005|     2

3189

In [11]:
filter_df = df.na.drop()

In [12]:
from pyspark.sql.functions import *

In [13]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))
# agg = exprs에 들어있는 계산식으로 dataframe 계산

unique_date_list = unique_date_df.collect()
new_date_list = []
for unique_date in unique_date_list:
    
    if unique_date[1] < 10 :
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + "0" + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
        
    else:
        if unique_date[2] < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + "0" +str(unique_date[2]) + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +str(unique_date[2]) + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [14]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+----------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|trade_price|    city|         sub_city|         dong| code|                           name|floor|exclusive_private_area|                           address|detailed_address|               date|
+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+----------------------------------+----------------+-------------------+
|2020|    1|  1|122|      2008|     15,000|경상남도|           양산시|       주남동|48330|                   신원아침도시|    9|               84.9879|           경상남도 양산시  주남동|            55=1|2020-01-01T00:00:00|
|2020|    1|  1| 49|      1998|      8,250|경상남도|           거창군|거창읍 김천리|48880|                       김천주공|    4|                 49

3189

In [15]:
added_date_df = added_date_df.withColumn("detailed_address", regexp_replace(col("detailed_address"),"=","-"))
added_date_df.show()

+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+------------------------------+-----+----------------------+----------------------------------+----------------+-------------------+
|year|month|day|_c0|build_year|trade_price|    city|         sub_city|         dong| code|                          name|floor|exclusive_private_area|                           address|detailed_address|               date|
+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+------------------------------+-----+----------------------+----------------------------------+----------------+-------------------+
|2020|    1|  1| 44|      2010|     46,900|경상남도|    창원시 의창구|       명서동|48121|             명곡두산위브1,2차|    4|                168.18|    경상남도 창원시 의창구  명서동|              27|2020-01-01T00:00:00|
|2020|    1|  1|106|      1998|     23,450|경상남도|           진주시|       신안동|48170|                     신안주공1|   14|                 84.98|     

In [16]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+----------------------------------+----------------+-----+
|                           address|detailed_address|count|
+----------------------------------+----------------+-----+
|             경상남도 양산시  명동|          1080-7|    2|
|     경상남도 양산시 물금읍 가촌리|          1304-1|    5|
| 경상남도 창원시 마산합포구  중...|           4-323|    1|
|           경상남도 거제시  아주동|            1682|    2|
|    경상남도 창원시 진해구  마천동|            99-3|    1|
|           경상남도 거제시  상동동|            1080|    4|
|           경상남도 거제시  양정동|             885|    3|
|         경상남도 진주시  충무공동|             302|   10|
|           경상남도 양산시  삼호동|             860|    4|
|             경상남도 김해시  외동|            1251|    4|
|경상남도 창원시 마산합포구  추산동|            36-2|    1|
|           경상남도 양산시  남부동|           602-1|    9|
|           경상남도 김해시  동상동|           495-7|    1|
|경상남도 창원시 마산회원구  양덕동|            64-1|    2|
|    경상남도 창원시 성산구  대방동|             375|    5|
|     경상남도 거창군 거창읍 대동리|           599-1|    2|
|     경상남도 거창군 거창읍 김천리|            99-1|    

1027

In [17]:
sum = new_df.select(sum("count"))
sum.show()

+----------+
|sum(count)|
+----------+
|      3189|
+----------+



In [18]:
unique_address_list = new_df.collect()
#print (unique_address_list[0][0])

In [19]:
import sys

def req_addr(address):
    data=0
    if '-' in address[1]:
        check = str(address[1]).split('-')[0]
        if address[1].split('-')[0].isdigit() == False or address[1].split('-')[1].isdigit() == False:
            print(address)
            print('주소에 글자섞임')
            data= 400
    if data == 0:
        response = kakao_req(address[0] + " " + address[1])
    else :
        return data
    if response == '403':
        print('빈배열 에러 ' + address[0] + " " + address[1])
        sleep(5)
        data= 3
        return data
    else :
        return response
        
    

In [40]:

location_list = []

# checking why no esult data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;
print (length)
addr2 =unique_address_list[:1]

for address in unique_address_list: 
    #full_address = (address[0] + " " + address[1]).encode('utf8')
    #print(full_address)
    response = req_addr(address)
    if response == 400 or response == 3:
        continue
    if response.status_code == 200:
        data = response.json()
        x = data['documents'][0]['address']['x']
        y = data['documents'][0]['address']['y']
        location_list.append([address[0],address[1],y,x])
    else: 
        print (response.json())
        count += 1
        #print (count)
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if index % 100 == 0:
        print (("%0.2f" % (index / length * 100.0)) + "% completed")
        
    index += 1.0

1027
9.74% completed
19.47% completed
29.21% completed
38.95% completed
48.69% completed
58.42% completed
Row(address='경상남도 김해시 주촌면 선지리', detailed_address='가-', count=25)
주소에 글자섞임
빈배열 에러 경상남도 진주시  가좌동 442
68.16% completed
77.90% completed
87.63% completed
97.37% completed


In [43]:
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+----------------------------------+----------------+----------------+----------------+
|                           address|detailed_address|        latitude|       longitude|
+----------------------------------+----------------+----------------+----------------+
|             경상남도 양산시  명동|          1080-7| 35.401943076242|129.171960642164|
|     경상남도 양산시 물금읍 가촌리|          1304-1| 35.314446557454|129.002328554626|
| 경상남도 창원시 마산합포구  중...|           4-323|35.1992115850417| 128.56819913429|
|           경상남도 거제시  아주동|            1682|34.8658073747537|128.685690028789|
|    경상남도 창원시 진해구  마천동|            99-3|35.1245776633035|128.783515848535|
|           경상남도 거제시  상동동|            1080|34.8654954190708|128.636960266036|
|           경상남도 거제시  양정동|             885| 34.889245877469|128.641769981746|
|         경상남도 진주시  충무공동|             302|35.1711835373942|128.144951382856|
|           경상남도 양산시  삼호동|             860|35.4063346745242|129.174321054854|
|             경상남도 김해시  외동|            1251

In [44]:
location.count()

1025

In [45]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [46]:
joined_df.show()
joined_df.count()

+---------------------------------+----------------+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|                          address|detailed_address|year|month|day|_c0|build_year|trade_price|    city|         sub_city|         dong| code|                           name|floor|exclusive_private_area|               date|        latitude|       longitude|
+---------------------------------+----------------+----+-----+---+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|          경상남도 거제시  아주동|            1682|2020|    1| 14| 14|      2012|     15,500|경상남도|           거제시|       아주동|48310|                    덕산아내1차|   11|               59.9652|2020-01-14T00:00:00|34.8658073747537|128.68569002878

3160

In [47]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+---------------------------------+----------------+----+-----+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|                          address|detailed_address|year|month|day|build_year|trade_price|    city|         sub_city|         dong| code|                           name|floor|exclusive_private_area|               date|        latitude|       longitude|
+---------------------------------+----------------+----+-----+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|          경상남도 거제시  아주동|            1682|2020|    1| 14|      2012|     15,500|경상남도|           거제시|       아주동|48310|                    덕산아내1차|   11|               59.9652|2020-01-14T00:00:00|34.8658073747537|128.685690028789|
|          경상

3160

In [48]:
final_apartment_df = drop_index_df.na.drop()
final_apartment_df = final_apartment_df.withColumn('trade_price',regexp_replace('trade_price',"\\,", ""))
final_apartment_df.show()
final_apartment_df.count()

+---------------------------------+----------------+----+-----+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|                          address|detailed_address|year|month|day|build_year|trade_price|    city|         sub_city|         dong| code|                           name|floor|exclusive_private_area|               date|        latitude|       longitude|
+---------------------------------+----------------+----+-----+---+----------+-----------+--------+-----------------+-------------+-----+-------------------------------+-----+----------------------+-------------------+----------------+----------------+
|          경상남도 거제시  아주동|            1682|2020|    1| 14|      2012|      15500|경상남도|           거제시|       아주동|48310|                    덕산아내1차|   11|               59.9652|2020-01-14T00:00:00|34.8658073747537|128.685690028789|
|          경상

3160

In [66]:
#final_apartment_df.write.format('com.databricks.spark.csv').save('mycsv.csv')
final_apartment_df.coalesce(1).write.csv('mycsv2.csv')